# Test-run for the ALALI package

First import the ALALI package.

In [1]:
import ALALI

Next, we can call ALALI's molecule class

In [2]:
proline = ALALI.molecule("OC(=O)C1CCCN1", "smi", "proline")

Now we've created and initialized the proline molecule. We can access a networkx graph of the molecule as a class attribute

In [3]:
graph = proline.graph

We can also output a .mol and .graphml file. If `printing` is set to `True`, the function will print mol text and visualize the networkx graph

In [4]:
proline.output(printing=False)

We can see that the graph is made of nodes=atoms and edges=bonds. We can list these atoms/bonds using networkx.

In [5]:
print("Atoms in form: (INDEX, {'symbol':ELEMENT})\n")
print(list(graph.nodes(data=True)))
print("\nBonds in form: (INDEX1, INDEX2)\n")
print(list(graph.edges))

Atoms in form: (INDEX, {'symbol':ELEMENT})

[(0, {'symbol': 'O'}), (1, {'symbol': 'C'}), (2, {'symbol': 'O'}), (3, {'symbol': 'C'}), (4, {'symbol': 'C'}), (5, {'symbol': 'C'}), (6, {'symbol': 'C'}), (7, {'symbol': 'N'}), (8, {'symbol': 'H'}), (9, {'symbol': 'H'}), (10, {'symbol': 'H'}), (11, {'symbol': 'H'}), (12, {'symbol': 'H'}), (13, {'symbol': 'H'}), (14, {'symbol': 'H'}), (15, {'symbol': 'H'}), (16, {'symbol': 'H'})]

Bonds in form: (INDEX1, INDEX2)

[(0, 1), (0, 8), (1, 2), (1, 3), (3, 4), (3, 7), (3, 9), (4, 5), (4, 10), (4, 11), (5, 6), (5, 12), (5, 13), (6, 7), (6, 14), (6, 15), (7, 16)]


Now we can create a pytket circuit where each qubit is an atom, and each CX is a bond between two atoms. The qubit indices match the atom indices from the networkx graph. 

In [12]:
circuit = ALALI.circuit_gen(graph)
print("Qubits in form: q_INDEX\n")
ALALI.print_circuit(circuit)

Qubits in form: q_INDEX

           ┌───┐          ┌───┐                                             »
 q_0: ──■──┤ X ├──■───────┤ X ├─────────────────────────────────────────────»
      ┌─┴─┐└─┬─┘  │       └─┬─┘┌───┐     ┌───┐                              »
 q_1: ┤ X ├──■────┼────■────┼──┤ X ├──■──┤ X ├──────────────────────────────»
      └───┘       │  ┌─┴─┐  │  └─┬─┘  │  └─┬─┘                              »
 q_2: ────────────┼──┤ X ├──┼────■────┼────┼────────────────────────────────»
                  │  └───┘  │       ┌─┴─┐  │       ┌───┐          ┌───┐     »
 q_3: ────────────┼─────────┼───────┤ X ├──■────■──┤ X ├──■───────┤ X ├─────»
                  │         │       └───┘     ┌─┴─┐└─┬─┘  │       └─┬─┘┌───┐»
 q_4: ────────────┼─────────┼─────────────────┤ X ├──■────┼────■────┼──┤ X ├»
                  │         │                 └───┘       │  ┌─┴─┐  │  └─┬─┘»
 q_5: ────────────┼─────────┼─────────────────────────────┼──┤ X ├──┼────■──»
                  │         │          

Now we can rout the circuit to a specific quantum computer topology. For example, the IBMQ Tokyo 20-qubit machine.

In [13]:
routed_circuit = ALALI.route_circuit(circuit, ALALI.ibm.ibmq_20_tokyo)

In [14]:
ibm_compatible_circuit = ALALI.circuit_to_ibm(routed_circuit)

Note that routing our circuit to the IBM qiskit architecture can result in errors due to BRIDGE gates. To resolve this issue, you can decompose the BRIDGE/SWAP gates into CXs, and then route the circuit.

In [15]:
ibm_compatible_circuit = ALALI.circuit_to_ibm(ALALI.decompose_circuit(routed_circuit, 1))

In [16]:
print(ibm_compatible_circuit)

                   ┌───┐                                                       »
 node_0: ──────────┤ X ├──■────────────────────────────────────────────────────»
         ┌───┐     └─┬─┘┌─┴─┐     ┌───┐                                        »
 node_1: ┤ X ├──■────■──┤ X ├──■──┤ X ├────────────────────────────────────────»
         └─┬─┘  │       └───┘┌─┴─┐└─┬─┘          ┌───┐     ┌───┐               »
 node_2: ──┼────┼────────────┤ X ├──■────■───────┤ X ├──■──┤ X ├───────────────»
           │    │            └───┘       │       └─┬─┘┌─┴─┐└─┬─┘               »
 node_3: ──┼────┼────────────────────────┼─────────┼──┤ X ├──■─────────────────»
           │    │                        │         │  └───┘                    »
 node_4: ──┼────┼────────────────────────┼─────────┼───────────────────────────»
           │    │  ┌───┐                 │         │                           »
 node_5: ──┼────┼──┤ X ├──■──────────────┼─────────┼───────────────────────────»
           │  ┌─┴─┐└─┬─┘┌─┴─

Now we have a circuit that matches our molecule, routed to a specific IBM architecture.